## Step 1: Import Required Libraries

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt


## Step 2: Load and Normalize the Fashion MNIST Dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

## Step 3: Resize Images and Convert to 3 Channels

In [ ]:
def resize_and_repeat(image):
    image_resized = tf.image.resize(image, (96, 96))
    image_resized = tf.repeat(image_resized, 3, axis=-1)
    return image_resized

## Step 4: Preprocess Data Using tf.data.Dataset

In [ ]:
def preprocess_data(x_data, y_data):
    dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
    dataset = dataset.map(lambda x, y: (resize_and_repeat(x), y))
    dataset = dataset.batch(32).prefetch(tf.data.AUTOTUNE)
    return dataset
train_dataset = preprocess_data(x_train, y_train)
test_dataset = preprocess_data(x_test, y_test)

## Step 5: Load Pre-trained MobileNetV2 Model

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(96, 96, 3))

## Step 6: Create Feature Extractor Model

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
feature_extractor = Model(inputs=base_model.input, outputs=x)

## Step 7: Extract Features Using CNN

In [ ]:
def extract_features(dataset):
    features = []
    labels = []
    for batch_images, batch_labels in dataset:
        batch_features = feature_extractor.predict(batch_images)
        features.append(batch_features)
        labels.append(batch_labels)
    return np.vstack(features), np.concatenate(labels)
train_features, y_train_extracted = extract_features(train_dataset)
test_features, y_test_extracted = extract_features(test_dataset)

## Step 8: Train SVM Classifier

In [ ]:
svm_clf = SVC(kernel='linear', C=1.0)
svm_clf.fit(train_features, y_train_extracted)

## Step 9: Make Predictions

In [ ]:
y_pred = svm_clf.predict(test_features)

## Step 10: Evaluate Model Performance

In [ ]:
accuracy = accuracy_score(y_test_extracted, y_pred)
print(f'Model Accuracy: {accuracy:.4f}')
print('\nClassification Report:\n', classification_report(y_test_extracted, y_pred))
print('\nConfusion Matrix:\n', confusion_matrix(y_test_extracted, y_pred))

## Step 11: Visualize Predictions

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(10, 5))
for ax, image, label, pred in zip(axes.ravel(), x_test[:10], y_test[:10], y_pred[:10]):
    ax.set_axis_off()
    ax.imshow(image, cmap='gray')
    ax.set_title(f'Label: {label}\nPred: {pred}')
plt.show()